# Experiment 07: Final

 
### The notebook is divided into the following main sections:
1. Imports and configuration parameters
2. Tile generation (sampling of tiles from the satellite images)
3. Tile input pipelines (`tf.dataset` objects reading tiles from disk)
4. Building of models
5. Pretraining with L1 loss
6. Build the full ESRGAN model
7. GAN-training with L1 + Percep + GAN loss
8. Inspection of results

Training history is logged with TensorBoard.

## 1. Imports and configuration parameters

In [1]:
from modules.helpers import *
from modules.tile_generator import *
from modules.matlab_metrics import *
from modules.image_utils import *
from modules.tile_input_pipeline import *
from modules.models import *
from modules.evaluation import *

from modules.logging import *
from modules.train import *

import time

CPU = False
if CPU:
    # Set CPU as available physical device
    tf.config.set_visible_devices([], 'GPU')
else:
    # Check GPUs and enable dynamic GPU memory use:",
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            for gpu in gpus:
                # Prevent TensorFlow from allocating all memory of all GPUs:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
### MAIN SETTINGS ###############################################################################################
EXPERIMENT_NAMES = ['e07-8', 'e07-4']

# Select experiment variation to be run in THIS notebook:
EXPERIMENT = EXPERIMENT_NAMES[1]

# Turn on and off certain time consuming processes in the notebook:
GENERATE_TILES = False   # This should only be done once in experiment 01. All variations will read from the same
TILE_DENSITY_MAPS = False  # Loops through all tiles and compute density maps of where tiles have been sampled
CALCULATE_STATS = False  # Loops through all tiles and calculate mean and sd. Used for scaling
PRE_BUILD = True          # Step 1 of the training process
PRETRAIN = False          # Step 1 of the training process
GAN_BUILD = True          # Step 1 of the training process
GAN_TRAIN = False         # Step 2 of the training process
BICUBIC_EVALUATE_LAST = False
PRE_EVALUATE_LAST = False
GAN_EVALUATE_LAST = False
PRE_EVALUATE_HISTORY = False
GAN_EVALUATE_HISTORY = False
HR_EVALUATE = False  # Compute Ma and NIQE for PAN instead of SR!

# Load metadata dataframe "meta" from repository root. 
# This dataframe keeops track of images and is used and updated throughout the notebook
meta = load_meta_pickle_csv('.', 'metadata_df', from_pickle=True)
#################################################################################################################

### PATHS #######################################################################################################
DATA_PATH = 'data/toulon-laspezia'
DATA_PATH_TILES = 'data/toulon-laspezia-tiles/e07'
DATA_PATH_TILES_P = {'train': DATA_PATH_TILES + '/train', 
                     'val': DATA_PATH_TILES + '/val', 
                     'test': DATA_PATH_TILES + '/test'}
LOGS_DIR = 'logs/' # Path to tensorboard logs and model checkpoint saves
LOGS_EXP_DIR = LOGS_DIR + EXPERIMENT
#################################################################################################################

### TILE GENERATION #############################################################################################
SENSORS_GENERATE = ['WV02', 'GE01']
AREAS_GENERATE = ['La_Spezia', 'Toulon']
meta = subset_by_areas_sensor(meta, areas=AREAS_GENERATE, sensors=SENSORS_GENERATE)
print('Sensors to generate tiles from:', SENSORS_GENERATE)
print('Areas to generate tiles from:', AREAS_GENERATE)

### CONVERT WV02 VAL IMAGES TO TRAIN!!! ##########################
def val_to_train(meta):
    meta.loc[(meta['train_val_test'] == 'val') 
             & (meta['sensorVehicle'] == 'WV02') , 'train_val_test'] = 'train'
    return meta
meta = val_to_train(meta)
##################################################################

# Count images in partitions (train/val/test):
N_IMAGES_TOTAL = count_images(meta)
N_IMAGES = {'train': count_images_in_partition(meta, 'train'), 
            'val': count_images_in_partition(meta, 'val'), 
            'test': count_images_in_partition(meta, 'test')}
assert N_IMAGES_TOTAL == sum(N_IMAGES.values())  # Verify that different ways of counting adds up
print('Number of images in partitions', N_IMAGES)
print('Total number of images:', N_IMAGES_TOTAL)

TILES_PER_M2 = {'train': 10.0,  ################################# <--- The oversampling of patches
                'val': 2.0, 
                'test': 2.0}

# Settings for whether to send every tile generated through a sea and cloud classifier
# This is useful if images consist of a lot of sea and clouds and you want to reduce the number of tiles
# with such monotone and less meaningful content. Classifier is trained on 2500 labeled tiles of various sizes
# where only tiles COMPLETELY covered by sea and/or clouds have been labelled "cloud/sea". 
# Validation accuracy around 0.95
CLOUD_SEA_REMOVAL = True
CLOUD_SEA_WEIGHTS_PATH = 'models/cloud-sea-classifier/cloudsea-effb0-augm-bicubic-pan-0.0005--200-0.127841.h5'
# Cutoff at inference time. Tiles with (quasi)-prob higher than cutoff will be classified as cloud and or sea:
CLOUD_SEA_PRED_CUTOFF = 0.95
# Setting to keep a certain proportion of cloud/sea tiles through the filter:
CLOUD_SEA_KEEP_RATE = 0.10

# GE01 images has some slight variations in resolution 0.5 +-0.05 m per pixel while WV02 is fixed at 0.5m
# Setting this to True will resize to as close as possible to 0.5m
# Not used in this notebook, but function is ready for use in module tile_generator.py
RESIZE_TO_PIXEL_SIZE = False
if RESIZE_TO_PIXEL_SIZE:
    RESIZE_RESAMPLING_METHOD = 'nearest'  # 'nearest', 'bicubic', 'bilinear'
    NEW_PIXEL_SIZE_PAN = 0.5
    RESIZE_DIR = DATA_PATH + '-resized'
    
# Data augmentation
AUGMENT_FLIP = True # both up/down and left/right flips
AUGMENT_ROTATE = True # 90 degree rotations
#################################################################################################################

### SENSORS AND AREA EXPERIMENT SELECTION #######################################################################
# Sensors used in which experiment variation
SENSORS_EXP = {'e07-8': {'train': 'WV02', 'test': ['WV02']}, 
               'e07-6': {'train': 'WV02', 'test': ['WV02']}, 
               'e07-4': {'train': 'WV02', 'val': ['GE01'], 'test': ['WV02', 'GE01']}, 
               'e07-3': {'train': 'WV02', 'val': ['GE01'], 'test': ['WV02', 'GE01']}}
SENSORS = SENSORS_EXP[EXPERIMENT]

# Areas used in which experiment variation
AREAS_EXP = {'e07-8': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e07-6': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e07-4': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}, 
             'e07-3': {'train': AREAS_GENERATE, 'val': AREAS_GENERATE, 'test': AREAS_GENERATE}}
AREAS = AREAS_EXP[EXPERIMENT]
#################################################################################################################

### TILE DIMENSIONS #############################################################################################
# Note larger size of val and test. This is needed for sensible calculation of Ma, NIQE and PI calculation
SR_FACTOR = 4
MS_SIZE = {'train': 32, 'val': 128, 'test': 128}
PAN_SIZE = {'train': MS_SIZE['train'] * SR_FACTOR, 
            'val': MS_SIZE['val'] * SR_FACTOR, 
            'test': MS_SIZE['test'] * SR_FACTOR}
print('MS (LR) tile size:', MS_SIZE)
print('PAN (HR) tile size:', PAN_SIZE)
print('SR factor:', SR_FACTOR)
#################################################################################################################

### BAND (CHANNEL) CONFIGURATIONS ###############################################################################
# This is the essence of experiment 01
# Selection of bands is done in the tile input pipeline

# Selecting bands from the 8 bands of WV02:
WV02_FULL_BAND_CONFIG = get_sensor_bands('WV02', meta)
WV02_EXP_BAND_CONFIGS = {'e07-8': WV02_FULL_BAND_CONFIG,                          # 8 (all) bands
                         'e07-6': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 6 bands (BGYR+RE+NIR)
                                   if k not in ['Coastal', 'NIR2']}, 
                         'e07-4': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 4 bands (BGR+NIR)
                                   if k in ['Blue', 'Green', 'Red', 'NIR']},
                         'e07-3': {k:v for (k,v) in WV02_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                   if k in ['Blue', 'Green', 'Red']}}
MS_BANDS_WV02_CONFIG = WV02_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e07-8':
    # We set this to 'all' in order to not pass e01-8 tiles through a band selection function (no reason to)
    MS_BANDS_WV02_IDXS = 'all' 
else:
    # For the other experiment variations we need lists of indices of the bands to be selected
    MS_BANDS_WV02_IDXS = list(MS_BANDS_WV02_CONFIG.values())

N_MS_BANDS = len(MS_BANDS_WV02_CONFIG.values()) # The number of MS bands in this experiment variation

# Selecting bands from the 4 bands of GE01:
GE01_FULL_BAND_CONFIG = get_sensor_bands('GE01', meta)                            
GE01_EXP_BAND_CONFIGS = {'e07-8': {None: None},                                   # not enough bands in GE01
                         'e07-6': {None: None},                                    # not enough bands in GE01
                         'e07-4': GE01_FULL_BAND_CONFIG,                           # 4 (all) bands (BGR+NIR)
                         'e07-3': {k:v for (k,v) in GE01_FULL_BAND_CONFIG.items()  # 3 bands (BGR)
                                   if k not in ['NIR']}}
MS_BANDS_GE01_CONFIG = GE01_EXP_BAND_CONFIGS[EXPERIMENT]
if EXPERIMENT == 'e07-4':
    MS_BANDS_GE01_IDXS = 'all'
else:
    MS_BANDS_GE01_IDXS = list(MS_BANDS_GE01_CONFIG.values())
print('MS (LR) Band Config WV02:', MS_BANDS_WV02_CONFIG)
print('MS (LR) Band Config GE01:', MS_BANDS_GE01_CONFIG)

N_PAN_BANDS = 1 # Obviously only 1 panchromatic band
#################################################################################################################

### MODEL PARAMETERS ############################################################################################
BATCH_SIZE = {'train': 16, 'val': 8, 'test': 8}
print('Batch sizes:', BATCH_SIZE)

# RRDB Generator Model parameters 
N_BLOCKS = 16 # Deeper means potential to capture more complex relationships, at the cost of training time
N_FILTERS = 64 # Baseline setting that is not tinkered with in this repository
#################################################################################################################

### PRETRAINING SETTINGS ########################################################################################
PRE_EPOCHS = 400
PRE_TRAIN_STEPS = 1000  # per epoch
PRE_VAL_STEPS = 0     # per epoch
print('Pretraining - Total steps:', PRE_EPOCHS * PRE_TRAIN_STEPS)

# Number of batches to save every epoch in TensorBoard
TRAIN_N_BATCHES_SAVE = 1
VAL_N_BATCHES_SAVE = 1

# Optimizer settings:
PRETRAIN_LOSS = 'l1'    # Official
PRETRAIN_LR = 5e-5      # Tuned and found stable for this particular experiment
#PRETRAIN_LR = 0.0002   # Official
PRETRAIN_BETA_1 = 0.9   # Official
PRETRAIN_BETA_2 = 0.999 # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations
#################################################################################################################

### GAN TRAINING SETTINGS #######################################################################################
GAN_EPOCHS = 400
GAN_TRAIN_STEPS = 1000
GAN_VAL_STEPS = 0
# Proportion of val batches that will go through ma and niqe metric calculation
# MA_NIQE_PROPORTION = 0.04  # The calculation is very time consuming
MA_NIQE_PROPORTION = 1  # The calculation is very time consuming
print('GAN training - Total steps:', GAN_EPOCHS * GAN_TRAIN_STEPS)

# Weights for each loss in the composite loss function
G_LOSS_PIXEL_W = 0.01       # Official
G_LOSS_PERCEP_W = 1.0       # Official
G_LOSS_GENERATOR_W = 0.005  # Official

# Optimizer settings:
#GAN_G_LR = 1e-4 # Official
#GAN_D_LR = 1e-4 # Official
GAN_G_LR = 2e-5
GAN_D_LR = 2e-5
G_BETA_1, D_BETA_1 = 0.9, 0.9      # Official
G_BETA_2, D_BETA_2 = 0.999, 0.999  # Official
# Note: Official implementation also uses stepwise learning rate scheduler. 
# This is avoided here as it is deemed not central to the experiment to "squeeze" out last performance and it 
# complicates comparisons between experiment variations

# Path to the pretraining weights that is the starting point of GAN training:
PRETRAIN_WEIGHTS_DIRS = {'e07-4': LOGS_EXP_DIR + '/models/' + 'e07-4-pre_20210401-212113/'
                        }
PRETRAIN_WEIGHTS_DIR = PRETRAIN_WEIGHTS_DIRS[EXPERIMENT]
PRETRAIN_WEIGHTS_PATH = PRETRAIN_WEIGHTS_DIR + EXPERIMENT + '-pre-400.h5'

# Path to the gan-training weights that will be 
GAN_WEIGHTS_DIRS = {'e07-4': LOGS_EXP_DIR + '/models/' + 'e07-4-gan_20210404-102643/'
                   }
GAN_WEIGHTS_DIR = GAN_WEIGHTS_DIRS[EXPERIMENT]
GAN_WEIGHTS_PATH = GAN_WEIGHTS_DIR + EXPERIMENT + '-gan-G-399.h5'
#################################################################################################################

### MATLAB METRICS ##############################################################################################
# Calculate Ma, NIQE and Perceptual Index (PI) metrics on the validation set(s) during GAN training:
# PI was metric used in PIRM2018 competition https://github.com/roimehrez/PIRM2018
METRIC_MA = True
METRIC_NIQE = True
if METRIC_MA and METRIC_NIQE:
    METRIC_PI = True
else:
    METRIC_PI = False

# The number of pixels to be shaved off the border of the tile before calculating Ma/NIQE/PI (ignore border effects)
SHAVE_WIDTH = 4 # Official (as used in PIRM2018 evaluation)
# Ma/NIQE/PI calculation is done with official matlab repositories through MATLAB Engine API for Python
MATLAB_PATH = 'modules/matlab' # path to repositories
#################################################################################################################

### EVALUTAION ##################################################################################################
if PRE_EVALUATE_LAST or GAN_EVALUATE_LAST:
    METRIC_MA = True
    METRIC_NIQE = True
    if METRIC_MA and METRIC_NIQE:
        METRIC_PI = True
    else:
        METRIC_PI = False
        

EVAL_STEPS_PER_EPOCH = 'all'
EVAL_N_EPOCHS = 400
#EVAL_SENSOR = 'WV02'
EVAL_PER_IMAGE = True
    
if PRE_EVALUATE_HISTORY or GAN_EVALUATE_HISTORY:
    METRIC_MA = False
    METRIC_NIQE = True
    if METRIC_MA and METRIC_NIQE:
        METRIC_PI = True
    else:
        METRIC_PI = False
        
#if PRE_EVALUATE_HISTORY:
#    EVAL_WEIGHTS_DIR = PRETRAIN_WEIGHTS_DIR
#    EVAL_FIRST_STEP = 1
#    EVAL_PREFIX = EXPERIMENT + '-pre-'
#elif GAN_EVALUATE_HISTORY:
#    EVAL_WEIGHTS_DIR = GAN_WEIGHTS_DIR
#    EVAL_FIRST_STEP = 0
#    EVAL_PREFIX = EXPERIMENT + '-gan-'
    
print('MATLAB Metrics:')
print('Ma:', METRIC_MA)
print('NIQE:', METRIC_NIQE)
print('Perceptual Index (PI):', METRIC_PI)

Sensors to generate tiles from: ['WV02', 'GE01']
Areas to generate tiles from: ['La_Spezia', 'Toulon']
Number of images in partitions {'train': 32, 'val': 9, 'test': 21}
Total number of images: 62
MS (LR) tile size: {'train': 32, 'val': 128, 'test': 128}
PAN (HR) tile size: {'train': 128, 'val': 512, 'test': 512}
SR factor: 4
MS (LR) Band Config WV02: {'Blue': 1, 'Green': 2, 'Red': 4, 'NIR': 6}
MS (LR) Band Config GE01: {'Blue': 0, 'Green': 1, 'Red': 2, 'NIR': 3}
Batch sizes: {'train': 16, 'val': 8, 'test': 8}
Pretraining - Total steps: 400000
GAN training - Total steps: 400000
MATLAB Metrics:
Ma: True
NIQE: True
Perceptual Index (PI): True


## 2. Tile generation

### 2.1 Image resizing

Function `resize_sat_img_to_new_pixel_size` available in `modules.tile_generator`. Not used in this notebook

### 2.2 Tile allocation

We allocate `n_tiles` to each satellite image in proportion to the area covered by the satellite image. We adjust `n_tiles` by the argument `tiles_per_m2`. If `tiles_per_m2=1.0` then `n_tiles` is set deterministically to a value so that a square meter of satellite image is expected to be covered by `1.0` tile.

In [4]:
if GENERATE_TILES:
    meta = allocate_tiles_by_expected(meta, 
                                      override_pan_pixel_size=RESIZE_TO_PIXEL_SIZE,
                                      by_partition=True, 
                                      tiles_per_m2_train_val_test=(TILES_PER_M2['train'], 
                                                                   TILES_PER_M2['val'], 
                                                                   TILES_PER_M2['test']),
                                      pan_tile_size_train_val_test=(PAN_SIZE['train'], 
                                                                    PAN_SIZE['val'], 
                                                                    PAN_SIZE['test']),
                                      new_column_name='n_tiles')
else:
    # Load meta dataframe that was updated at tile generation time
    meta = load_meta_pickle_csv(DATA_PATH_TILES, 'metadata_tile_allocation', from_pickle=True)

n_tiles = {'train': count_tiles_in_partition(meta, 'train'),
           'val': count_tiles_in_partition(meta, 'val'), 
           'test':  count_tiles_in_partition(meta, 'test')}
n_tiles_total = count_tiles(meta)
assert n_tiles_total == sum(n_tiles.values())
print('Number of tiles per partition:')
print(n_tiles)
print('Total number of tiles:', n_tiles_total)

Number of tiles per partition:
{'train': 935412, 'val': 3709, 'test': 9278}
Total number of tiles: 948399


### 2.3 Tile generation to disk

In [5]:
if GENERATE_TILES:
    meta = generate_all_tiles(meta, 
                              save_dir=DATA_PATH_TILES, 
                              sr_factor=SR_FACTOR, 
                              by_partition=True,
                              ms_tile_size_train_val_test=(MS_SIZE['train'], MS_SIZE['val'], MS_SIZE['test']), 
                              cloud_sea_removal=CLOUD_SEA_REMOVAL, 
                              cloud_sea_weights_path=CLOUD_SEA_WEIGHTS_PATH, 
                              cloud_sea_pred_cutoff=CLOUD_SEA_PRED_CUTOFF,
                              cloud_sea_keep_rate=CLOUD_SEA_KEEP_RATE,
                              save_meta_to_disk=True)

In [6]:
if TILE_DENSITY_MAPS:
    for row in meta.iterrows():
        img_uid = row[0]
        density = tile_density_map(DATA_PATH_TILES, 
                                   row[1], 
                                   pan_or_ms='pan',
                                   density_dtype='uint8',
                                   write_to_disk=True,
                                   write_dir=DATA_PATH_TILES + '/density-maps', 
                                   write_filename=img_uid)
    # Plot last density
    plt.imshow(density)

In [7]:
if CALCULATE_STATS:
    train_tiles_mean, train_tiles_sd = mean_sd_of_train_tiles(DATA_PATH_TILES, 
                                                              sample_proportion=1.0, 
                                                              write_json=True)
else:
    train_tiles_mean, train_tiles_sd = read_mean_sd_json(DATA_PATH_TILES)

Loaded mean 346.7 and sd 134.9 from json file @ data/toulon-laspezia-tiles/e07/train_mean_sd.json


## 3. Data input pipeline from disk

### 3.1 Training set

In [8]:
#SHUFFLE_BUFFER_SIZE = {'train': 90000,  # 100
#                       'val': 8000,  # 100
#                       'test': 8000}  # 100
#
SHUFFLE_BUFFER_SIZE = {'train': 100,  # 100
                       'val': 100,  # 100
                       'test': 100}  # 100

train_val_test = 'train'
sensor = SENSORS[train_val_test]
ds_train = {sensor: GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                   batch_size=BATCH_SIZE[train_val_test], 
                                   ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                   pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                   sensor=sensor,
                                   band_selection=MS_BANDS_WV02_IDXS, 
                                   mean_correction=train_tiles_mean,
                                   cache_memory=True,
                                   cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-' 
                                                  + train_val_test + '-' + sensor + '_cache'), 
                                   repeat=True, 
                                   shuffle=True, 
                                   shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test],
                                   augment_flip=True,
                                   augment_rotate=True
                                  )
           }
# Getting the scaled output range from the scaler. Needed to calculate PSNR and SSIM:
scaled_range = ds_train[sensor].get_scaler_output_range(print_ranges=True)

# Returning the actual tf.data.dataset object:
ds_train[sensor] = ds_train[sensor].get_dataset()
print(ds_train.keys())

Scaler ranges:
Input (uint) min, max: 0 2047
Input (uint) range: 2048
Output (float) range 1.2044782512000547
Output (float) min, max: -0.20389012705396095 1.0
dict_keys(['WV02'])


### 3.2 Validation set

In [9]:
# Validation set can have several sensors and is organized in a dictionary
# structure: ds_val = {sensor: dataset} ... ex: ds_val = {'WV02': dataset_with_only_WV02_images}
train_val_test = 'test' #### <- Special only for e07
ds_val = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_val[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                    batch_size=BATCH_SIZE[train_val_test], 
                                    ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                    pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                    sensor=sensor,
                                    band_selection=band_indices, 
                                    mean_correction=train_tiles_mean,
                                    cache_memory=True,
                                    cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                   + train_val_test + '-' + sensor + '_cache'), 
                                    repeat=True, 
                                    shuffle=True, 
                                    shuffle_buffer_size=SHUFFLE_BUFFER_SIZE[train_val_test])
    ds_val[sensor] = ds_val[sensor].get_dataset()
print(ds_val.keys())

dict_keys(['WV02', 'GE01'])


## 4. Build preliminary models

### 4.1 Bicubic baseline model

In [10]:
bicubic = build_deterministic_sr_model(upsample_factor=SR_FACTOR,
                                       resize_method='bicubic',
                                       loss='mean_absolute_error',
                                       metrics=('PSNR', 'SSIM'),
                                       scaled_range=scaled_range)

### 4.2 ESRGAN Generator model (pretrain version)

In [11]:
if PRE_BUILD:
    pretrain_model =  build_generator(pretrain_or_gan='pretrain', 
                                      pretrain_learning_rate=PRETRAIN_LR, 
                                      pretrain_loss_l1_l2=PRETRAIN_LOSS,
                                      pretrain_beta_1=PRETRAIN_BETA_1, 
                                      pretrain_beta_2=PRETRAIN_BETA_2, 
                                      pretrain_metrics=('PSNR', 'SSIM'),
                                      scaled_range=scaled_range, 
                                      n_channels_in=N_MS_BANDS, 
                                      n_channels_out=N_PAN_BANDS, 
                                      height_width_in=None,  # None will make network image size agnostic
                                      n_filters=N_FILTERS, 
                                      n_blocks=N_BLOCKS)
    # pretrain_model.summary()

## 5. Pretraining with L1 loss

In [12]:
if PRETRAIN:
    history = pretrain_esrgan(generator=pretrain_model,
                              ds_train_dict=ds_train,
                              epochs=PRE_EPOCHS,
                              steps_per_epoch=PRE_TRAIN_STEPS,
                              initial_epoch=0,
                              validate=True,
                              ds_val_dict=ds_val,
                              val_steps=PRE_VAL_STEPS,
                              model_name=EXPERIMENT + '-pre',
                              tag=EXPERIMENT,
                              log_tensorboard=True,
                              tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                              save_models=True,
                              models_save_dir=LOGS_EXP_DIR + '/models',
                              save_weights_only=True,
                              log_train_images=True,
                              n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                              log_val_images=True,
                              n_val_image_batches=VAL_N_BATCHES_SAVE)

## 6. Build the full ESRGAN Model

In [13]:
if GAN_BUILD:
    gan_model = build_esrgan_model(PRETRAIN_WEIGHTS_PATH,
                                   n_channels_in=N_MS_BANDS, 
                                   n_channels_out=N_PAN_BANDS, 
                                   n_filters=N_FILTERS, 
                                   n_blocks=N_BLOCKS, 
                                   pan_shape=(PAN_SIZE['train'], PAN_SIZE['train'], N_PAN_BANDS),
                                   G_lr=GAN_G_LR, 
                                   D_lr=GAN_D_LR, 
                                   G_beta_1=G_BETA_1, 
                                   G_beta_2=G_BETA_2, 
                                   D_beta_1=D_BETA_1, 
                                   D_beta_2=D_BETA_2,
                                   G_loss_pixel_w=G_LOSS_PIXEL_W, 
                                   G_loss_pixel_l1_l2='l1',
                                   G_loss_percep_w=G_LOSS_PERCEP_W, 
                                   G_loss_percep_l1_l2='l1', 
                                   G_loss_percep_layer=54,
                                   G_loss_percep_before_act=True,
                                   G_loss_generator_w=G_LOSS_GENERATOR_W,
                                   metric_reg=False, 
                                   metric_ma=METRIC_MA, 
                                   metric_niqe=METRIC_NIQE, 
                                   ma_niqe_proportion=MA_NIQE_PROPORTION,
                                   matlab_wd_path='modules/matlab',
                                   scale_mean=train_tiles_mean, 
                                   scaled_range=scaled_range, 
                                   shave_width=SHAVE_WIDTH, 
                                   metric_ma_niqe_hr=HR_EVALUATE)

Starting matlab.engine ...
matlab.engine started


## 7. GAN training

In [14]:
if GAN_TRAIN:
    history = gan_train_esrgan(esrgan_model=gan_model,
                               ds_train_dict=ds_train,
                               epochs=GAN_EPOCHS,
                               steps_per_epoch=GAN_TRAIN_STEPS,
                               initial_epoch=0,
                               validate=True,
                               ds_val_dict=ds_val,
                               val_steps=GAN_VAL_STEPS,
                               model_name=EXPERIMENT + '-gan',
                               tag=EXPERIMENT,
                               log_tensorboard=True,
                               tensorboard_logs_dir=LOGS_EXP_DIR + '/tb',
                               save_models=True,
                               models_save_dir=LOGS_EXP_DIR + '/models',
                               save_weights_only=True,
                               log_train_images=True,
                               n_train_image_batches=TRAIN_N_BATCHES_SAVE,
                               log_val_images=True,
                               n_val_image_batches=VAL_N_BATCHES_SAVE)

## Predict single

## 8. Evaluation

### 8.1 Data input pipelines for final evaluation

The pipeline is modified to include the file paths of the tiles/patches so that it is possible to log performance metrics for individual files and by extension for individual satellite images.

#### 8.1.2 Test set

In [ ]:
train_val_test = 'test'
ds_test = {}
for sensor in SENSORS[train_val_test]:
    if sensor == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    elif sensor == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    ds_test[sensor] = GeotiffDataset(tiles_path=DATA_PATH_TILES_P[train_val_test], 
                                     batch_size=BATCH_SIZE[train_val_test], 
                                     ms_tile_shape=(MS_SIZE[train_val_test], MS_SIZE[train_val_test], N_MS_BANDS), 
                                     pan_tile_shape=(PAN_SIZE[train_val_test], PAN_SIZE[train_val_test], N_PAN_BANDS),
                                     sensor=sensor,
                                     band_selection=band_indices, 
                                     mean_correction=train_tiles_mean,
                                     cache_memory=False,
                                     cache_file=str(DATA_PATH_TILES + '/ds_' + EXPERIMENT + '-'
                                                    + train_val_test + '-' + sensor + '_filepath_cache'), 
                                     repeat=False, 
                                     shuffle=False, 
                                     shuffle_buffer_size=0, 
                                     include_file_paths=True)
    ds_test[sensor] = ds_test[sensor].get_dataset()
print(ds_test.keys())

### 8.2 Evaluate last epoch

In [ ]:
if PRE_EVALUATE_LAST or GAN_EVALUATE_LAST or HR_EVALUATE:
    val_or_test = 'test'

    if PRE_EVALUATE_LAST and not GAN_EVALUATE_LAST:
        PRE_GAN = ['pre']
    if not PRE_EVALUATE_LAST and GAN_EVALUATE_LAST:
        PRE_GAN = ['gan']
    if HR_EVALUATE:
        PRE_GAN = ['HR']

    print(PRE_GAN, SENSORS[val_or_test])
    for pre_gan in PRE_GAN:
        
        for sensor in SENSORS[val_or_test]:
            # if sensor == 'GE01':
            #     continue
            if sensor == 'GE01':
                band_indices = MS_BANDS_GE01_IDXS
            elif sensor == 'WV02':
                band_indices = MS_BANDS_WV02_IDXS
            if pre_gan == 'pre':
                gan_model.G.load_weights(PRETRAIN_WEIGHTS_PATH)
            elif pre_gan == 'gan':
                gan_model.G.load_weights(GAN_WEIGHTS_PATH)
            elif pre_gan == 'HR':
                # load pretrain weights. The model does not use the weights during HR evaluation
                gan_model.G.load_weights(PRETRAIN_WEIGHTS_PATH)
            else:
                raise NotImplementedError

            print(pre_gan, sensor)
            start = time.time()
            results_df = esrgan_evaluate(model=gan_model, 
                                         dataset=ds_test[sensor], 
                                         steps='all', 
                                         per_image=True, 
                                         write_csv=True,
                                         csv_path=str(LOGS_EXP_DIR + '/csv/' + 'final_epoch-' 
                                                      + pre_gan + '-' + val_or_test + '-' + sensor + '.csv'), 
                                         verbose=1
                                        )
            end = time.time()
            print(str((end - start) / 60), 'minutes')

### 8.3 Evaluate every kth epoch

In [ ]:
if PRE_EVALUATE_HISTORY or GAN_EVALUATE_HISTORY:
    val_or_test = 'test'
    
    # Computing Ma is 100x more time consuming than anything else. It is not interesting to measure this for pretraining
    if METRIC_MA:
        PRE_GAN = ['gan']
    else:
        PRE_GAN = ['pre', 'gan']

    for pre_gan in PRE_GAN:
        for sensor in SENSORS[val_or_test]:
            if sensor == 'GE01':
                band_indices = MS_BANDS_GE01_IDXS
            elif sensor == 'WV02':
                band_indices = MS_BANDS_WV02_IDXS
            if pre_gan == 'pre':
                model_weights_dir = PRETRAIN_WEIGHTS_DIR
                eval_first_step = 1
                eval_prefix = EXPERIMENT + '-pre-'
            else:
                model_weights_dir = GAN_WEIGHTS_DIR
                eval_first_step = 0
                eval_prefix = EXPERIMENT + '-gan-'

            esrgan_epoch_evaluator(gan_model,
                                   model_weights_dir=model_weights_dir,
                                   model_weight_prefix=eval_prefix,
                                   dataset=ds_test[sensor],
                                   n_epochs=EVAL_N_EPOCHS,
                                   first_epoch=eval_first_step,
                                   steps_per_epoch=EVAL_STEPS_PER_EPOCH,
                                   k_epoch=25,
                                   csv_dir=LOGS_EXP_DIR + '/csv/' + val_or_test + '-' + sensor, 
                                       per_image=EVAL_PER_IMAGE, 
                                   verbose=0)

### 8.4 Comparison plots

In [ ]:
def list_all_ms_tiles(tiles_dir):
    if isinstance(tiles_dir, str):
        tiles_dir = pathlib.Path(tiles_dir)
    ms_tiles_paths = [p for p in tiles_dir.glob('**/ms/*.tif')]
    return ms_tiles_paths

def esrgan_predict_multiple(
    ms_tiles_paths, 
    results_dir, 
    esrgan_generator_model,
    bicubic_model,
    pretrain_weights_path, 
    gan_weights_path,
    wv02_band_indices,
    ge01_band_indices,
    mean_correction,
    sr_factor, 
    geotiff_or_png='png'):
    if isinstance(results_dir, str):
        results_dir = pathlib.Path(results_dir)
    results_dir.mkdir(parents=True, exist_ok=True)
    
    if geotiff_or_png == 'png':
        output_dtype = 'uint8'
    elif geotiff_or_png == 'geotiff':
        output_dtype = 'uint16' 
    else:
        raise ValueError
    
    count = 0
    for ms_tile_path in ms_tiles_paths:
        results_img_dir = results_dir.joinpath(ms_tile_path.parents[1].stem)
        results_img_dir.mkdir(parents=True, exist_ok=True)
        if 'WV02' in str(ms_tile_path):
            sensor = 'WV02'
            band_indices = wv02_band_indices
        elif 'GE01' in str(ms_tile_path):
            sensor = 'GE01'
            band_indices = ge01_band_indices
        else:
            raise ValueError
        for pre_gan in ['pre', 'gan', 'bicubic']:
            if pre_gan == 'pre':
                predict_model = esrgan_generator_model
                predict_model.load_weights(pretrain_weights_path)
            elif pre_gan == 'gan':
                predict_model = esrgan_generator_model
                predict_model.load_weights(gan_weights_path)
            else:
                predict_model = bicubic_model
            sr_img = esrgan_predict(
                model=predict_model,
                ms_img_path=ms_tile_path,
                result_dir=results_img_dir,
                sensor=sensor,
                band_indices=band_indices,
                mean_correction=mean_correction,
                pre_or_gan=pre_gan,
                sr_factor=sr_factor,
                copy_pan_img=True,
                pan_img_path=None,
                output_dtype=output_dtype,
                geotiff_or_png=geotiff_or_png)
        count += 1
        print(count, 'Saved ms, pre, gan and pan image for patch', results_img_dir, ms_tile_path.name)

In [ ]:
ms_tiles_paths = list_all_ms_tiles('data/toulon-laspezia-tiles/e07/test/')
print(len(ms_tiles_paths))

esrgan_predict_multiple(
    ms_tiles_paths=ms_tiles_paths, 
    results_dir='results/thesis-appendix/', 
    esrgan_generator_model=pretrain_model,
    bicubic_model=bicubic,
    pretrain_weights_path=PRETRAIN_WEIGHTS_PATH, 
    gan_weights_path=GAN_WEIGHTS_PATH,
    wv02_band_indices=MS_BANDS_WV02_IDXS,
    ge01_band_indices=MS_BANDS_GE01_IDXS,
    mean_correction=train_tiles_mean,
    sr_factor=SR_FACTOR, 
    geotiff_or_png='png')

In [ ]:

    
IMG = 'GE01_Toulon 2014_10_16_011651185010_0'
#IMG = 'GE01_La_Spezia_2009_09_25_011651186010_0'
#IMG = 'GE01_La_Spezia_2019_01_03_011651196010_0'
#IMG = 'GE01_La_Spezia_2013_07_23_011651202010_0'
IMG = 'GE01_Toulon 2010_06_08_011651191010_0'
MS_IMG_PATH = 'data/toulon-laspezia-tiles/e07/test/' + IMG + '/ms/00227.tif'

RESULTS_DIR = 'results/thesis-appendix/' + IMG
SENSOR = 'GE01'
predict_model = pretrain_model
for pre_gan in ['pre', 'gan', 'bicubic']:
    if SENSOR == 'GE01':
        band_indices = MS_BANDS_GE01_IDXS
    elif SENSOR == 'WV02':
        band_indices = MS_BANDS_WV02_IDXS
    if pre_gan == 'pre':
        predict_model = pretrain_model
        predict_model.load_weights(PRETRAIN_WEIGHTS_PATH)
    elif pre_gan == 'gan':
        predict_model = pretrain_model
        predict_model.load_weights(GAN_WEIGHTS_PATH)
    else:
        predict_model = bicubic
    sr_img = esrgan_predict(model=predict_model, 
                            ms_img_path=MS_IMG_PATH, 
                            result_dir=RESULTS_DIR, 
                            sensor=SENSOR, 
                            band_indices=band_indices, 
                            mean_correction=train_tiles_mean, 
                            pre_or_gan=pre_gan,
                            sr_factor=SR_FACTOR, 
                            copy_pan_img=True,
                            pan_img_path=None, 
                            output_dtype='uint16', 
                            geotiff_or_png='png')

### 8.5 Epoch plots

In [26]:
#IMG = 'GE01_Toulon 2014_10_16_011651185010_0'
IMG = 'WV02_La_Spezia_2019_06_03_011650596010_0'
#IMG = 'GE01_La_Spezia_2009_09_25_011651186010_0'
#IMG = 'GE01_La_Spezia_2019_01_03_011651196010_0'
#IMG = 'GE01_La_Spezia_2013_07_23_011651202010_0'
#IMG = 'GE01_Toulon 2010_06_08_011651191010_0'
MS_IMG_PATH = 'data/toulon-laspezia-tiles/e07/train/' + IMG + '/ms/00615.tif'
MS_IMG_PATH = 'data/toulon-laspezia-tiles/e07/train/' + IMG + '/ms/00808.tif'

def esrgan_predict_train_history(
    ms_tile_path, 
    results_dir, 
    esrgan_generator_model,
    pretrain_weights_dir, 
    gan_weights_dir,
    model,
    wv02_band_indices,
    ge01_band_indices,
    mean_correction,
    sr_factor, 
    geotiff_or_png='png'):
    
    if geotiff_or_png == 'png':
        output_dtype = 'uint8'
    elif geotiff_or_png == 'geotiff':
        output_dtype = 'uint16' 
    else:
        raise ValueError
    
    if 'WV02' in str(ms_tile_path):
        sensor = 'WV02'
        band_indices = wv02_band_indices
    elif 'GE01' in str(ms_tile_path):
        sensor = 'GE01'
        band_indices = ge01_band_indices
    else:
        raise ValueError
        
    for i in range(1, 400):
        model_weights = pretrain_weights_dir + '/' + model + '-pre-' + str(i).zfill(2) + '.h5'
        print(model_weights)
        esrgan_generator_model.load_weights(model_weights)
        sr_img = esrgan_predict(
                    model=esrgan_generator_model,
                    ms_img_path=ms_tile_path,
                    result_dir=results_dir,
                    sensor=sensor,
                    band_indices=band_indices,
                    mean_correction=mean_correction,
                    pre_or_gan='pre',
                    sr_factor=sr_factor,
                    copy_pan_img=True,
                    pan_img_path=None,
                    output_dtype=output_dtype,
                    geotiff_or_png=geotiff_or_png, 
                    suffix='-' + str(i).zfill(3))
    for i in range(400):
        model_weights = gan_weights_dir + '/' + model + '-gan-G-' + str(i) + '.h5'
        print(model_weights)
        esrgan_generator_model.load_weights(model_weights)
        sr_img = esrgan_predict(
                    model=esrgan_generator_model,
                    ms_img_path=ms_tile_path,
                    result_dir=results_dir,
                    sensor=sensor,
                    band_indices=band_indices,
                    mean_correction=mean_correction,
                    pre_or_gan='gan',
                    sr_factor=sr_factor,
                    copy_pan_img=True,
                    pan_img_path=None,
                    output_dtype=output_dtype,
                    geotiff_or_png=geotiff_or_png, 
                    suffix='-' + str(i).zfill(3))
        
esrgan_predict_train_history(
    ms_tile_path=MS_IMG_PATH, 
    results_dir='results/history/', 
    esrgan_generator_model=pretrain_model,
    pretrain_weights_dir='logs/e07-4/models/e07-4-pre_20210401-212113', 
    gan_weights_dir='logs/e07-4/models/e07-4-gan_20210404-102643',
    model='e07-4',
    wv02_band_indices=MS_BANDS_WV02_IDXS,
    ge01_band_indices=MS_BANDS_GE01_IDXS,
    mean_correction=train_tiles_mean,
    sr_factor=SR_FACTOR, 
    geotiff_or_png='png')
        

logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-01.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-02.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-03.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-04.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-05.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-06.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-07.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-08.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-09.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-10.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-11.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-12.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-13.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-14.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-15.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-16.h5
logs/e07-4/models/e07-4-pre_20210401-212

logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-137.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-138.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-139.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-140.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-141.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-142.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-143.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-144.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-145.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-146.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-147.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-148.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-149.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-150.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-151.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-152.h5
logs/e07-4/models/e07-4-

logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-272.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-273.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-274.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-275.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-276.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-277.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-278.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-279.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-280.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-281.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-282.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-283.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-284.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-285.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-286.h5
logs/e07-4/models/e07-4-pre_20210401-212113/e07-4-pre-287.h5
logs/e07-4/models/e07-4-

logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-7.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-8.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-9.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-10.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-11.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-12.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-13.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-14.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-15.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-16.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-17.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-18.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-19.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-20.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-21.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-22.h5
logs/e07-4/

logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-139.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-140.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-141.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-142.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-143.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-144.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-145.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-146.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-147.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-148.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-149.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-150.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-151.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-152.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-153.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G

logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-270.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-271.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-272.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-273.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-274.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-275.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-276.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-277.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-278.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-279.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-280.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-281.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-282.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-283.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G-284.h5
logs/e07-4/models/e07-4-gan_20210404-102643/e07-4-gan-G

In [ ]:
t = 501
str(t).zfill(2)

print('1' + None)